In [1]:
import logging
import sys
from functools import partial
from pathlib import Path

import pandas as pd
from datasets import load_dataset
from omegaconf import OmegaConf
from peft import PeftModel
from transformers import (
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

dataset_name = "kamel-usp/aes_enem_dataset"
dataset_split = "JBCS2025"
cache_dir = "/tmp/"
base_model = "microsoft/phi-4"
fine_tuned_model_id = "kamel-usp/jbcs2025_phi4-balanced-C1"
grade_index = int(fine_tuned_model_id[-1]) - 1
print(f"Grade index being used: {grade_index}")

Grade index being used: 0


In [2]:
parent_dir = str(Path(".").resolve().parent) + "/scripts"
sys.path.append(str(parent_dir))

In [3]:
from preprocess import load_tokenizer, tokenize_dataset

from metrics.metrics import compute_metrics
from models.fine_tuning_models.model_types_enum import ModelTypesEnum

In [4]:
model_type = ModelTypesEnum.PHI4_CLASSIFICATION_LORA.value
model_type

'phi4_classification_lora'

In [5]:
dataset = load_dataset(
    dataset_name,
    dataset_split,
    cache_dir=cache_dir,
)

# Load the tokenizer
tokenizer = load_tokenizer(
    model_type,
    base_model,
    cache_dir=cache_dir,
)

tokenized_dataset = tokenize_dataset(
    dataset,
    tokenizer,
    text_column="essay_text",
    grade_index=grade_index,
    model_type=model_type,
    logger=logging.getLogger(),
)

In [6]:
NUM_LABELS = 6
if model_type in [
    ModelTypesEnum.LLAMA31_CLASSIFICATION_LORA.value,
    ModelTypesEnum.PHI35_CLASSIFICATION_LORA.value,
    ModelTypesEnum.PHI4_CLASSIFICATION_LORA.value,
]:
    base_model = AutoModelForSequenceClassification.from_pretrained(
        base_model, num_labels=NUM_LABELS
    )
    model = PeftModel.from_pretrained(base_model, fine_tuned_model_id)
else:
    model = AutoModelForSequenceClassification.from_pretrained(
        fine_tuned_model_id, cache_dir=cache_dir, num_labels=NUM_LABELS
    )
model

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-4 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Phi3ForSequenceClassification(
      (model): Phi3Model(
        (embed_tokens): Embedding(100352, 5120, padding_idx=100349)
        (layers): ModuleList(
          (0-39): 40 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
         

In [7]:
experiment_config = OmegaConf.create({"experiments": {"model": {"type": model_type}}})

In [8]:
compute_metrics_partial = partial(compute_metrics, cfg=experiment_config)
PER_DEVICE_EVAL = 1

training_args = TrainingArguments(
    output_dir="test_trainer",
    do_eval=True,
    per_device_eval_batch_size=PER_DEVICE_EVAL,
    bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics_partial,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 700.00 MiB. GPU 0 has a total capacity of 47.54 GiB of which 570.62 MiB is free. Process 3272099 has 46.94 GiB memory in use. Of the allocated memory 46.68 GiB is allocated by PyTorch, and 2.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
if model_type in [
    ModelTypesEnum.LLAMA31_CLASSIFICATION_LORA.value,
    ModelTypesEnum.PHI35_CLASSIFICATION_LORA.value,
    ModelTypesEnum.PHI4_CLASSIFICATION_LORA.value,
]:
    model.config.pad_token_id = model.config.eos_token_id
assert model.config.pad_token_id is not None

In [ ]:
eval_results = trainer.evaluate(tokenized_dataset["test"])

In [ ]:
pd.DataFrame.from_dict(eval_results, orient="index").T

,eval_loss,eval_model_preparation_time,eval_accuracy,eval_RMSE,eval_QWK,eval_HDIV,eval_Macro_F1,eval_Micro_F1,eval_Weighted_F1,eval_TP_0,...,eval_TN_4,eval_FP_4,eval_FN_4,eval_TP_5,eval_TN_5,eval_FP_5,eval_FN_5,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,1.938086,0.0182,0.275362,60.910959,0.413047,0.086957,0.232232,0.275362,0.264557,9.0,...,98.0,8.0,28.0,0.0,133.0,2.0,3.0,91.3716,1.51,0.755


In [ ]:
result = 0
num_classes = 0
for label in range(NUM_LABELS):
    try:
        result += (
            2
            * eval_results[f"eval_TP_{label}"]
            / (
                2 * eval_results[f"eval_TP_{label}"]
                + eval_results[f"eval_FP_{label}"]
                + eval_results[f"eval_FN_{label}"]
            )
        )
        num_classes+=1
    except (ZeroDivisionError, KeyError):
        continue
assert result / num_classes == eval_results["eval_Macro_F1"]

In [ ]:
print(f"""
metrics:
          - name: Macro F1
            type: f1
            value: {eval_results["eval_Macro_F1"]}
          - name: QWK
            type: qwk
            value: {eval_results["eval_QWK"]}
          - name: Weighted Macro F1
            type: f1
            value: {eval_results["eval_Weighted_F1"]}""")


metrics:
          - name: Macro F1
            type: f1
            value: 0.23223169343896355
          - name: QWK
            type: qwk
            value: 0.41304694561114375
          - name: Weighted Macro F1
            type: f1
            value: 0.2645574847932973


In [ ]:
columns_to_use = [
    "eval_accuracy",
    "eval_RMSE",
    "eval_QWK",
    "eval_Macro_F1",
    "eval_Weighted_F1",
    "eval_Micro_F1",
    "eval_HDIV",
]
report_df = pd.DataFrame.from_dict(eval_results, orient="index").T
report_df = report_df[columns_to_use].T
print(report_df.rename(columns={0: "test_data"}).to_markdown())

|                  |   test_data |
|:-----------------|------------:|
| eval_accuracy    |   0.275362  |
| eval_RMSE        |  60.911     |
| eval_QWK         |   0.413047  |
| eval_Macro_F1    |   0.232232  |
| eval_Weighted_F1 |   0.264557  |
| eval_Micro_F1    |   0.275362  |
| eval_HDIV        |   0.0869565 |
